# Reccomendation task

##Mounting the drive to read the data

In [0]:
import pandas as pd
import ast
import operator

from datetime import datetime
import json
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import itertools
import scipy as sp
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import normalize
from scipy import sparse



from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

!java -version
!sudo update-alternatives --config java
!java -version/

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

#importing libraries for spark mllib 
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [0]:
#This code will read the embedding vectors of each business of our subset into a dictionary with key as the business ID and values as embeddings
st=""
with open('/content/drive/My Drive/businessEmbeddings2.txt') as f:
  for line in f:
        st=st+line

#Converting json compatible format
business_embeddings = ast.literal_eval(st)

In [0]:
#Importing Business data
line_count = len(open("/content/drive/My Drive/yelp_dataset/business.json").readlines())
business_ids, names, addresses, cities, states, postal_codes, latitudes, longitudes, stars, review_counts, is_open, attributes, categories, hours = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
with open("/content/drive/My Drive/yelp_dataset/business.json") as f:
  for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        business_ids += [blob["business_id"]]
        names += [blob["name"]]
        addresses += [blob["address"]]
        cities += [blob["city"]]
        states += [blob["state"]]
        postal_codes += [blob["postal_code"]]
        latitudes += [blob["latitude"]]
        longitudes += [blob["longitude"]]
        stars += [blob["stars"]]
        review_counts += [blob["review_count"]]
        is_open += [blob["is_open"]]
        attributes += [blob["attributes"]]
        categories += [blob["categories"]]
        hours += [blob["hours"]]
businesses = pd.DataFrame(
    {"business_id": business_ids, "name": names, "address": addresses, "city": cities, "state": states, "postal_code": postal_codes, "latitude": latitudes, "longitude": longitudes, "rating": stars, "review_count": review_counts, "is_open": is_open, "attributes": attributes, "categories": categories, "hours": hours}
)

100%|██████████| 192609/192609 [00:03<00:00, 48604.11it/s]


In [0]:
businesses.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,rating,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


## Encoding business data - Categories and Attribues

In [0]:
bus_2 = businesses.loc[businesses['city'] == 'Las Vegas']

In [0]:
mask_restaurants = bus_2['categories'].str.contains('Restaurants')
mask_food = bus_2['categories'].str.contains('Food')
restaurants_and_food = bus_2[mask_restaurants & mask_food]
restaurants_and_food_business_ids = restaurants_and_food.business_id.to_list()
bus_2['isRestaurant'] = bus_2.business_id.isin(restaurants_and_food_business_ids).astype(int)
bus_3 = bus_2[bus_2.loc[:,'isRestaurant'] == 1]
#data2 = data2.iloc[:,:36]
bus_3.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


(2615, 15)

In [0]:
# "name" column name is ambiguous with df.name - change it
business_df = bus_3.rename(columns = {'name': 'BusinessName'})

In [0]:
categorylist = business_df['categories'].tolist()
categories = [st.split(', ') for st in categorylist]
mlb = MultiLabelBinarizer()
categoryarray = mlb.fit_transform(categories)
names = mlb.classes_
cat_df = pd.DataFrame(categoryarray,columns=names)
cat_df
merged_data = pd.concat([business_df.reset_index(),cat_df],axis=1)

In [0]:
merged_data_1 = merged_data.join(pd.DataFrame(merged_data['attributes'].to_dict()).T)
merged_data_1.head(1)

,index,business_id,BusinessName,address,city,state,postal_code,latitude,longitude,rating,review_count,is_open,attributes,categories,hours,isRestaurant,CategoryList,Acai Bowls,Accessories,Active Life,Adult Entertainment,Afghan,African,Air Duct Cleaning,Aircraft Repairs,Airports,Airsoft,American (New),American (Traditional),Amusement Parks,Appliances,Appliances & Repair,Arcades,Argentine,Armenian,Art Galleries,Arts & Entertainment,Asian Fusion,Audio/Visual Equipment Rental,Auto Repair,...,Women's Clothing,Wraps,Yelp Events,BikeParking,BusinessParking,BusinessAcceptsCreditCards,GoodForKids,HappyHour,Alcohol,GoodForMeal,OutdoorSeating,WiFi,RestaurantsAttire,Caters,RestaurantsReservations,RestaurantsTakeOut,RestaurantsDelivery,Ambience,RestaurantsGoodForGroups,NoiseLevel,HasTV,RestaurantsPriceRange2,ByAppointmentOnly,BusinessAcceptsBitcoin,GoodForDancing,RestaurantsTableService,Music,BestNights,WheelchairAccessible,DogsAllowed,BYOBCorkage,DriveThru,Smoking,CoatCheck,Corkage,DietaryRestrictions,AcceptsInsurance,Open24Hours,BYOB,RestaurantsCounterService
0,25,tstimHoMcYbkSC4eBA1wEg,Maria's Mexican Restaurant & Bakery,6055 E Lake Mead Blvd,Las Vegas,NV,89156,36.195615,-115.040529,4.5,184,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Mexican, Restaurants, Patisserie/Cake Shop, Fo...","{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0'...",1,"[Mexican, Restaurants, Patisserie/Cake Shop, F...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,True,"{'garage': False, 'street': False, 'validated'...",True,True,False,u'beer_and_wine',"{'dessert': False, 'latenight': False, 'lunch'...",False,u'no',u'casual',False,False,True,False,"{'romantic': False, 'intimate': False, 'classy...",True,'average',True,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# columns with non-boolean categorical values:
cols_to_split = ['Alcohol', 'BYOBCorkage', 'NoiseLevel', 'RestaurantsAttire', 'Smoking', 'WiFi']
new_cat = pd.concat([pd.get_dummies(merged_data_1[col], prefix=col, prefix_sep='_') for col in cols_to_split], axis=1)
# keep all columns (not n-1) because 0's for all of them indicates that the data was missing (useful info)
merged_data_2 = pd.concat([merged_data_1, new_cat], axis=1)
merged_data_2.drop(cols_to_split, inplace=True, axis=1)
merged_data_2.head()

,index,business_id,BusinessName,address,city,state,postal_code,latitude,longitude,rating,review_count,is_open,attributes,categories,hours,isRestaurant,CategoryList,Acai Bowls,Accessories,Active Life,Adult Entertainment,Afghan,African,Air Duct Cleaning,Aircraft Repairs,Airports,Airsoft,American (New),American (Traditional),Amusement Parks,Appliances,Appliances & Repair,Arcades,Argentine,Armenian,Art Galleries,Arts & Entertainment,Asian Fusion,Audio/Visual Equipment Rental,Auto Repair,...,AcceptsInsurance,Open24Hours,BYOB,RestaurantsCounterService,Alcohol_'beer_and_wine',Alcohol_'full_bar',Alcohol_'none',Alcohol_u'beer_and_wine',Alcohol_u'full_bar',Alcohol_u'none',BYOBCorkage_'no',BYOBCorkage_'yes_corkage',BYOBCorkage_'yes_free',NoiseLevel_'average',NoiseLevel_'loud',NoiseLevel_'quiet',NoiseLevel_'very_loud',NoiseLevel_u'average',NoiseLevel_u'loud',NoiseLevel_u'quiet',NoiseLevel_u'very_loud',RestaurantsAttire_'casual',RestaurantsAttire_'dressy',RestaurantsAttire_None,RestaurantsAttire_u'casual',RestaurantsAttire_u'dressy',RestaurantsAttire_u'formal',Smoking_'no',Smoking_'outdoor',Smoking_None,Smoking_u'no',Smoking_u'outdoor',Smoking_u'yes',WiFi_'free',WiFi_'no',WiFi_'paid',WiFi_None,WiFi_u'free',WiFi_u'no',WiFi_u'paid'
0,25,tstimHoMcYbkSC4eBA1wEg,Maria's Mexican Restaurant & Bakery,6055 E Lake Mead Blvd,Las Vegas,NV,89156,36.195615,-115.040529,4.5,184,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Mexican, Restaurants, Patisserie/Cake Shop, Fo...","{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0'...",1,"[Mexican, Restaurants, Patisserie/Cake Shop, F...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,75,kANF0dbeoW34s2vwh6Umfw,Dairy Queen,6125 Spring Mountain Rd,Las Vegas,NV,89146,36.125031,-115.225620,2.0,33,0,"{'RestaurantsPriceRange2': '1', 'Ambience': '{...","Fast Food, Food, Restaurants, Ice Cream & Froz...",None,1,"[Fast Food, Food, Restaurants, Ice Cream & Fro...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,173,v7ZFEEqJBg_r8NCwHRP_2A,Bistro Divino,241 W Charleston Blvd,Las Vegas,NV,89102,36.158264,-115.157967,4.5,3,0,"{'BusinessParking': '{'garage': False, 'street...","Food, Pizza, Wine Bars, Bars, Restaurants, Nig...",None,1,"[Food, Pizza, Wine Bars, Bars, Restaurants, Ni...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,198,4n81G-pmC3rfhmaPsbwYKg,"Vince Neil's Tatuado | Eat, Drink, Party",360 E Tropicana Ave,Las Vegas,NV,89169,36.102050,-115.156288,4.5,3,1,None,"Bars, Sports Bars, Nightlife, Beer, Wine & Spi...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,"[Bars, Sports Bars, Nightlife, Beer, Wine & Sp...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,206,PWjgwphd2prhtqTPWwLbXw,Stage Deli Of Las Vegas,"3500 Las Vegas Blvd S, Ste E11",Las Vegas,NV,89109,36.117327,-115.176501,3.5,28,0,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Sandwiches, Restaurants, Delis, Desserts, Food","{'Monday': '6:30-3:30', 'Tuesday': '6:30-3:30'...",1,"[Sandwiches, Restaurants, Delis, Desserts, Food]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
cols = merged_data_2.columns

In [0]:
newCol = list()


for index, row in merged_data_2.iterrows():
  for c in cols:
    if c == 'business_id' or c == 'index' or c == 'BusinessName':
      continue
    elif row[c] == 0 or row[c] == 1:
      continue
    else:
      newCol.append(c)
  break
print(newCol)

['address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'rating', 'review_count', 'attributes', 'categories', 'hours', 'CategoryList', 'BikeParking', 'BusinessParking', 'BusinessAcceptsCreditCards', 'GoodForKids', 'HappyHour', 'GoodForMeal', 'OutdoorSeating', 'Caters', 'RestaurantsReservations', 'RestaurantsTakeOut', 'RestaurantsDelivery', 'Ambience', 'RestaurantsGoodForGroups', 'HasTV', 'RestaurantsPriceRange2', 'ByAppointmentOnly', 'BusinessAcceptsBitcoin', 'GoodForDancing', 'RestaurantsTableService', 'Music', 'BestNights', 'WheelchairAccessible', 'DogsAllowed', 'DriveThru', 'CoatCheck', 'Corkage', 'DietaryRestrictions', 'AcceptsInsurance', 'Open24Hours', 'BYOB', 'RestaurantsCounterService']


In [0]:
merged_data_4 = merged_data_2.drop(columns = newCol)
merged_data_4.head()

,index,business_id,BusinessName,is_open,isRestaurant,Acai Bowls,Accessories,Active Life,Adult Entertainment,Afghan,African,Air Duct Cleaning,Aircraft Repairs,Airports,Airsoft,American (New),American (Traditional),Amusement Parks,Appliances,Appliances & Repair,Arcades,Argentine,Armenian,Art Galleries,Arts & Entertainment,Asian Fusion,Audio/Visual Equipment Rental,Auto Repair,Automotive,Bagels,Bakeries,Bangladeshi,Barbeque,Bars,Beauty & Spas,Bed & Breakfast,Beer,Beer Bar,Beer Garden,Beer Gardens,...,Wineries,Women's Clothing,Wraps,Yelp Events,Alcohol_'beer_and_wine',Alcohol_'full_bar',Alcohol_'none',Alcohol_u'beer_and_wine',Alcohol_u'full_bar',Alcohol_u'none',BYOBCorkage_'no',BYOBCorkage_'yes_corkage',BYOBCorkage_'yes_free',NoiseLevel_'average',NoiseLevel_'loud',NoiseLevel_'quiet',NoiseLevel_'very_loud',NoiseLevel_u'average',NoiseLevel_u'loud',NoiseLevel_u'quiet',NoiseLevel_u'very_loud',RestaurantsAttire_'casual',RestaurantsAttire_'dressy',RestaurantsAttire_None,RestaurantsAttire_u'casual',RestaurantsAttire_u'dressy',RestaurantsAttire_u'formal',Smoking_'no',Smoking_'outdoor',Smoking_None,Smoking_u'no',Smoking_u'outdoor',Smoking_u'yes',WiFi_'free',WiFi_'no',WiFi_'paid',WiFi_None,WiFi_u'free',WiFi_u'no',WiFi_u'paid'
0,25,tstimHoMcYbkSC4eBA1wEg,Maria's Mexican Restaurant & Bakery,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,75,kANF0dbeoW34s2vwh6Umfw,Dairy Queen,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,173,v7ZFEEqJBg_r8NCwHRP_2A,Bistro Divino,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,198,4n81G-pmC3rfhmaPsbwYKg,"Vince Neil's Tatuado | Eat, Drink, Party",1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,206,PWjgwphd2prhtqTPWwLbXw,Stage Deli Of Las Vegas,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
merged_data_4 = merged_data_4.drop(columns = ['index', 'BusinessName'])
merged_data_4.head()

,business_id,is_open,isRestaurant,Acai Bowls,Accessories,Active Life,Adult Entertainment,Afghan,African,Air Duct Cleaning,Aircraft Repairs,Airports,Airsoft,American (New),American (Traditional),Amusement Parks,Appliances,Appliances & Repair,Arcades,Argentine,Armenian,Art Galleries,Arts & Entertainment,Asian Fusion,Audio/Visual Equipment Rental,Auto Repair,Automotive,Bagels,Bakeries,Bangladeshi,Barbeque,Bars,Beauty & Spas,Bed & Breakfast,Beer,Beer Bar,Beer Garden,Beer Gardens,Belgian,Beverage Store,...,Wineries,Women's Clothing,Wraps,Yelp Events,Alcohol_'beer_and_wine',Alcohol_'full_bar',Alcohol_'none',Alcohol_u'beer_and_wine',Alcohol_u'full_bar',Alcohol_u'none',BYOBCorkage_'no',BYOBCorkage_'yes_corkage',BYOBCorkage_'yes_free',NoiseLevel_'average',NoiseLevel_'loud',NoiseLevel_'quiet',NoiseLevel_'very_loud',NoiseLevel_u'average',NoiseLevel_u'loud',NoiseLevel_u'quiet',NoiseLevel_u'very_loud',RestaurantsAttire_'casual',RestaurantsAttire_'dressy',RestaurantsAttire_None,RestaurantsAttire_u'casual',RestaurantsAttire_u'dressy',RestaurantsAttire_u'formal',Smoking_'no',Smoking_'outdoor',Smoking_None,Smoking_u'no',Smoking_u'outdoor',Smoking_u'yes',WiFi_'free',WiFi_'no',WiFi_'paid',WiFi_None,WiFi_u'free',WiFi_u'no',WiFi_u'paid'
0,tstimHoMcYbkSC4eBA1wEg,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,kANF0dbeoW34s2vwh6Umfw,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,v7ZFEEqJBg_r8NCwHRP_2A,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4n81G-pmC3rfhmaPsbwYKg,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,PWjgwphd2prhtqTPWwLbXw,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
businessDict = dict()
for index, row in merged_data_4.iterrows():
  businessDict[row['business_id']] = list()

businessList = merged_data_4.values.tolist()

In [0]:
i = 0
for key, value in businessDict.items():
  templist = businessList[i]
  templist.pop(0)
  businessDict[key] = templist
  i = i + 1
len(businessDict['tstimHoMcYbkSC4eBA1wEg'])

354

In [0]:
f = open("businessEmbeddings2.txt","w")
f.write( str(businessDict) )
f.close()


## Defining helper functions for getting reccomendations

In [0]:
#function to calculate hamming distance between lists
def hammin(l1,l2):
  l=list(map(operator.sub, l1, l2))
  #Generating absolute values
  ll=map(abs, l)
  return sum(ll)

In [0]:
def get_k_closest_neighbors(busi_id,kk):
  dist_list=[]
  busi_emb=business_embeddings[busi_id]
  for k,v in business_embeddings.items():
    if k!=busi_id: #For not calculating the distance between the business and itself
      d=hammin(busi_emb,v)
      dist_list.append((k,d))

  return sorted(dist_list, key = lambda x: x[1])[:kk]  

In [0]:
def get_recos(dfv):
  final_list={}
  c=0
  for index, row in dfv.iterrows():
    u_id= row['user_id']
    bu_id= row['business_id']
    p_t= row['predicted_threshold']

    if p_t==True:
      #This means the user has liked the last business he went to so let us reccomend similar businesses 
      final_list[u_id]=get_k_closest_neighbors(bu_id,5)

    else:
      c=c+1
      continue

  return final_list
    

## Generating Reccomendations for users using Business Similarity 

In [0]:
#last_business.csv has the columns user_id, business_id, and Predicted threshold of the last business the users of our subset have rated
#predicted threshold is either true or false, true if the user had above average experience with the last business he rated (rating>2.5), false otherwise (rating<2.5)
dfv=pd.read_csv("/content/drive/My Drive/last_business.csv")
fl= get_recos(dfv)

#Removing the distance from the reccomendation list of tuple, so the new dictionary will be like 'user_id':[reccomended_business1, reccomended_business2....]
#And not like 'user_id':[('reccomended_business1',distance1),('reccomended_business2',distance2)...]

lst_reccomendations={}
for k,v in fl.items():
  lst_reccomendations[k]= [i[0] for i in v]


## Generating Recommendation for users using ALS

In [0]:
city_name = "Las Vegas"
path = "/content/drive/My Drive/yelp_dataset/"
header = ['user_id', 'business_id', 'rating', 'date']
data3 = pd.read_csv(path + city_name + '_reviews_ratings_only.txt', sep='\t', names = header)

In [0]:
labels, uniques = pd.factorize(data3['user_id'])
data3 = data3.assign(user_id_encoded = labels)
cols = data3.columns.tolist()
cols = cols[-1:] + cols[:-1]
data3 = data3[cols]

labels, uniques = pd.factorize(data3['business_id'])
data3 = data3.assign(business_id_encoded = labels)
cols = data3.columns.tolist()
cols = cols[-1:] + cols[:-1]
data3 = data3[cols]
data3.head()

,business_id_encoded,user_id_encoded,user_id,business_id,rating,date
0,0,0,nMeCE5-xsdleyxYuNZ_7rA,oxwGyA17NL6c5t1Etg5WgQ,3.0,2012-02-29 21:52:43
1,1,1,FIk4lQQu1eTe2EpzQ4xhBA,8mIrX_LrOnAqWsB5JrOojQ,4.0,2011-11-30 02:11:15
2,2,2,3CJUJILq7CLHk_9OrvpvQg,I4Nr-MVc26qWr08-S3Q1ow,4.0,2016-01-17 05:26:22
3,3,3,C_hUvw2z0R-Rv0yZb6QCZA,dm6sO_Y8JdKTE1ZM955yug,5.0,2014-12-17 19:04:33
4,4,4,fxrQSMhHlBv6eJVRdchG9Q,IMLrj2klosTFvPRLv56cng,4.0,2014-10-11 03:50:27


In [0]:
data3['review_rating'] = data3.index
test_df = data3.sort_values(['user_id', 'date'], ascending=[True, False])
test_df = test_df.groupby('user_id').first().reset_index()

train_df = data3[~data3['review_rating'].isin(test_df['review_rating'])]

In [0]:
training = spark.createDataFrame(train_df)
test = spark.createDataFrame(test_df)

In [0]:
#Testing it on the test data
start = datetime.now()
print("Start Time =", start)
als = ALS(maxIter=10, regParam=0.05, 
          userCol="user_id_encoded", itemCol="business_id_encoded", ratingCol="rating",
          coldStartStrategy="drop",
          implicitPrefs=False, rank=10)
model = als.fit(training)
end = datetime.now()
print("End Time =", end)
time_1 = (end - start).total_seconds()
print(time_1)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
rmse_1 = rmse
print("Root-mean-square error = " + str(rmse))

#calculating r2 as the secondary metric
evaluator = RegressionEvaluator(metricName="r2", labelCol="rating",
                                predictionCol="prediction")

r2 = evaluator.evaluate(predictions)
r2_1 = r2
print("Root-squared error = " + str(r2))

Start Time = 2019-12-20 05:07:15.871198
End Time = 2019-12-20 05:07:37.888894
22.017696
Root-mean-square error = 1.4376757808556304
Root-squared error = -0.08265463488011826


In [0]:
#city_name = "Las Vegas"
path = "/content/drive/My Drive/Datasets/ap1.csv"
#header = ['user_id', 'business_id', 'rating', 'date']
predictedData = pd.read_csv(path)

In [0]:
predictedData.head(1) 

,Unnamed: 0,user_id,business_id,predicted,true,predicted_threshold,true_threshold
0,0,oDyEexUT09e3XhxIp0-Wxg,qsJkjQoM3Hdg6Gtqp2S9_w,3.850271,4.0,True,True


In [0]:
userListForALS = predictedData.loc[predictedData['predicted_threshold'] == False]
usersDf = pd.merge(userListForALS, data3, on = ['user_id', 'business_id'])

In [0]:
usersDf = usersDf[["user_id","user_id_encoded"]]

In [0]:
usersDf = usersDf.drop_duplicates()

In [0]:
users = test.select(als.getUserCol()).distinct()

In [0]:
userSubsetRecs = model.recommendForUserSubset(users, 5)

In [0]:
userSubsetRecs.show(1)

+---------------+--------------------+
|user_id_encoded|     recommendations|
+---------------+--------------------+
|           4519|[[8584, 5.647464]...|
+---------------+--------------------+
only showing top 1 row



In [0]:
userRecoPandas = userSubsetRecs.toPandas()

In [0]:
temp =[]
for i in userRecoPandas["recommendations"]:
  temp1=[]
  for j in i:
    temp1.append(j[0])
  temp.append(temp1)

In [0]:
userRecoPandas["recom"] = temp

In [0]:
userRecoPandas =userRecoPandas[["recom","user_id_encoded"]]

In [0]:
reco = pd.merge(usersDf,userRecoPandas,how="left",on="user_id_encoded")

In [0]:
reco = reco[["user_id","recom"]]

In [0]:
reco

,user_id,recom
0,XKpSkCt6yQnpoR1l9-jYTQ,"[6343, 456, 6118, 3805, 5823]"
1,wRY6wk5N6MRR6QVikwn17A,"[934, 8281, 4234, 10472, 7337]"
2,nysL4JwA8Y_iPYAo2kBNDA,"[8020, 9291, 3073, 4069, 9530]"
3,5owNSGo6RELDdzBTxCFaxQ,"[7722, 9051, 6055, 6075, 2055]"
4,6FyCo_qm9V8fDP3DeZgotg,"[2571, 203, 3977, 4976, 6238]"
...,...,...
206,dcpE4CUvtqLf63tRJgNqKQ,"[7246, 934, 7348, 3637, 1861]"
207,bnw9Oe3oMMOPRzkH71p-nA,"[2029, 9507, 9960, 6337, 9534]"
208,ntlvfPzc8eglqvk92iDIAw,"[4720, 3859, 7158, 1628, 4807]"
209,9u9fQhq0hynxjLQgZGpmSA,"[10593, 9051, 792, 2845, 6616]"


In [0]:
temp=[]
for r in reco["recom"]:
  temp1=[]
  for j in r:
    temp1.append(data3.loc[data3["business_id_encoded"]==j]["business_id"].unique()[0])
  temp.append(temp1)


In [0]:
reco["recommendations"] = temp

In [0]:
reco = reco[["user_id","recommendations"]]

In [0]:
reco

,user_id,recommendations
0,XKpSkCt6yQnpoR1l9-jYTQ,"[8peeO5aLwll6F-IC6IQagg, ZGSXoJUnnR-QO6_CTnLNv..."
1,wRY6wk5N6MRR6QVikwn17A,"[memEfE8NkM7iQ07ClDw0Jw, FNCJpSn0tL9iqoY3JC73q..."
2,nysL4JwA8Y_iPYAo2kBNDA,"[mny2GDCiv3HKr8xMr6fafQ, GQakJXDFLOIqdz9H1d58b..."
3,5owNSGo6RELDdzBTxCFaxQ,"[HI-j-I01o3emyU5MEdptyw, MBiQo_evqLa3pUDkh9rjW..."
4,6FyCo_qm9V8fDP3DeZgotg,"[oxyRSWO_IpIxgOn18ogD_g, slFIiXbpnqrBks33rbZwW..."
...,...,...
206,dcpE4CUvtqLf63tRJgNqKQ,"[qNa3Cx05uSTQV9FaDms-Cg, memEfE8NkM7iQ07ClDw0J..."
207,bnw9Oe3oMMOPRzkH71p-nA,"[uCGs0_5Sk0WFqyafDIs9NA, mYTem99eRslMJhWpfQOKG..."
208,ntlvfPzc8eglqvk92iDIAw,"[QoRPgmyMxb5hUuR1h258yQ, qSOZ3rgIWYSmf2kkeooh6..."
209,9u9fQhq0hynxjLQgZGpmSA,"[8u9lWKYiBneHzMHq3ReOYQ, MBiQo_evqLa3pUDkh9rjW..."


In [0]:
reco.to_csv('/content/drive/My Drive/yelp_dataset/'+"alsreco.csv")

In [0]:
dict1 = {}
for i,r in reco.iterrows():
  dict1.update({r["user_id"]:r["recommendations"]})

## Merging the reccomendations generated from ALS and our ensemble model

In [0]:
#Merging the Reccomendations for users that didn't like their last business from the ALS (Because for the users that did not like their last business, we are reccomending them top 5 businesses from the ALS model)
#This file has reccomendations for users that didn't like their last business as a dictionary

sts=""
with open('/content/drive/My Drive/userRecommedation.txt') as f:
  for line in f:
        sts=sts+line

#Converting json compatible format
balance_recos = ast.literal_eval(sts)

#Merging both the reccomendations
final_recos = {**lst_reccomendations, **balance_recos}

## Printing the catalogue coverage of the final reccomendations given to the users

In [0]:
def catalogue_coverage(dicti):
  lk=[]
  u=0
  for k,v in dicti.items():
    lk=lk+v

  #Unique businesses reccomended overall
  u=len(set(lk))
  #Overall unique no of businesses in subset
  l=len(business_embeddings)

  return (u/l)

print("Catalogue coverage of businesses:",catalogue_coverage(final_recos))

Catalogue coverage of businesses: 0.6764818355640535
